**Tools in LangChain:**

Tools are ways for the language model to use external functions or services, like a calculator or web search.

In [9]:
import os
from langchain.chat_models import init_chat_model

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

model = init_chat_model('groq:qwen/qwen3-32b')
response = model.invoke("What is the capital of France?")
response


AIMessage(content='<think>\nOkay, the user is asking, "What is the capital of France?" Let me start by recalling basic geography. France is a country in Western Europe, and its capital is a well-known city. I think it\'s Paris. But wait, maybe I should double-check to be sure.\n\nI remember that Paris is the capital, but sometimes there\'s confusion with other major cities like Lyon or Marseille. Let me think. The capital is usually the seat of government, and in France, that\'s definitely Paris. The government buildings, the Eiffel Tower, the Louvre—all located there. Yeah, that\'s right. \n\nWait, is there any chance it\'s a trick question? Like, sometimes countries have different capitals for different reasons. For example, some countries have moved their capitals. But France hasn\'t moved its capital. Paris has been the capital for a long time, even during the French Revolution. So I think it\'s safe to say Paris is the answer. \n\nAlso, checking historical context: after the Frenc

In [40]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # In a real implementation, this would call a weather API.
    return f"The current weather in {location} is sunny with a high of 25°C."

model_with_tools = model.bind_tools([get_weather])

In [ ]:
response = model_with_tools.invoke("What's the weather like in New York?")
# response.content
for tool_call in response.tool_calls:
    print(f"Tool : {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    # print(f"Tool called: {tool_call['name']} with arguments {tool_call['arguments']}")

''

# Tool Execution Loops

In [19]:
messages = [{'role': 'user', 'content': "What's the weather like in Pakistan"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

final_response = model_with_tools.invoke(messages)
print(final_response.text)



The current weather in Pakistan is sunny with a high of 25°C. It sounds like a pleasant day to be outdoors!


In [20]:
messages

[{'role': 'user', 'content': "What's the weather like in Pakistan"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Pakistan. I need to use the get_weather function for that. The function requires a location parameter. Pakistan is the location here, so I should call the function with "Pakistan" as the argument. Let me make sure the parameters are correctly formatted. Yep, it\'s a simple object with location. I\'ll return the tool call with the name get_weather and the arguments as location: Pakistan.\n', 'tool_calls': [{'id': 'skmemf4cf', 'function': {'arguments': '{"location":"Pakistan"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 155, 'total_tokens': 268, 'completion_time': 0.198986637, 'completion_tokens_details': {'reasoning_tokens': 89}, 'prompt_time': 0.007813684, 'prompt_tokens_details': None, 'queue_time': 0.011232096, 'total_time': 0